## A 3D Demo

The notebook shows how to extract SN-Graph out of a 3D volume.

In [ ]:
import trimesh
import numpy as np
from sn_graph import create_sn_graph, draw_sn_graph, visualize_3d_graph

In [ ]:
mesh = trimesh.load("../assets/cup_0085.off")
print("Displaying original mesh...")
mesh.show()

In [ ]:
# local voxelise allows to fill the inside of the mesh with positive values which is needed for sdf computation
voxels = trimesh.voxel.creation.local_voxelize(
    mesh, point=(0, 0, 0), radius=300, pitch=0.5, fill=True
)
print("Displaying voxelized mesh...")
voxels.show()

In [ ]:
binary_volume = np.array(voxels.matrix.astype(int))

graph = create_sn_graph(binary_volume, return_sdf=True, minimal_sphere_radius=2)
graph_volume = draw_sn_graph(*graph)

print(f"No vertices: {len(graph[0])}.")
print(f"No edges: {len(graph[1])}.")

In [ ]:
graph_voxels = trimesh.voxel.VoxelGrid(graph_volume).as_boxes()
print("Displaying voxelized graph...")
graph_voxels.show()

In [ ]:
scene = visualize_3d_graph(*graph)
print("Displaying graph with spheres (as a 3D mesh)...")
scene.show()

In [ ]:
vertices, edges, _ = graph
scene = visualize_3d_graph(vertices, edges)
print("Displaying graph without spheres (as a 3D mesh)...")
scene.show()